This is try to predict transembrane domaim from large set of data


In [1]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD
from math import log, exp

In [2]:
# A dictionary for parameters of single amino acid
# 'A': [Hydropathicity, side chain charge, polar, Interface Scale, pKa_sidechain, pI, Octanol Scale]
aa_info = {'A': [1.80, 0, 0, 0.17, 0.0, 6.0, 0.5],
           'C': [2.50, 0, 0, -0.24, 8.3, 5.1, 0],
           'D': [-3.5, -1, 1, 1.23, 3.9, 2.8, 3.64],
           'E': [-3.5, -1, 1, 0.0, 4.3, 3.2, 0.11],
           'F': [2.80, 0, 0, -1.13, 0.0, 5.5, -1.71],
           'G': [-0.4, 0, 0, 0, 0.0, 6.0, 1.15],
           'H': [-3.2, 0, -1, 0.17, 6.0, 7.6, 0.11],
           'I': [4.50, 0, 0, -0.31, 0.0, 6.0, -1.12],
           'K': [-3.9, 1, -1, 0.99, 10.5, 9.7, 2.8],
           'L': [3.80, 0, 0, -0.56, 0.0, 6.0, -1.25],
           'M': [1.90, 0, 0, -0.23, 0.0, 5.7, -0.67],
           'N': [-3.5, 0, 1, 0.42, 0.0, 5.4, 0.85],
           'P': [2.80, 0, 0, 0.45, 0.0, 6.3, 0.14],
           'Q': [-3.5, 0, 1, 0.58, 0.0, 5.7, 0.77],
           'R': [-4.5, 1, -1, 0.81, 12.5, 10.8, 1.81],
           'S': [-0.8, 0, 1, 0.13, 0.0, 5.6, 0.46],
           'T': [-0.7, 0, 1, 0.14, 0.0, 5.6, 0.25],
           'V': [4.20, 0, 0, 0.07, 0.0, 6.0, -0.46],
           'W': [-0.9, 0, 0, -1.85, 0.0, 5.9, -2.09],
           'Y': [-1.3, 0, 1, -0.94, 10.7, 5.7, -0.71]}

In [3]:
def parse(line):
    """ Extract labels and features from raw data
    
    :param line (string): single line from the input file, starts with 0 or 1, 
                 0 means that it is not transmenbrane domain, 1 means it is transmenbrane domain
                 20 charactors after the space, standing for 20 residues
        
    :return: LabeledPoint: labeled with 0 or 1, and the features calculated from the peptides sequence  
             [Hydropathicity, side chain charge, polar, MW, pKa_sidechain, pI]
             
    """
    allAminoAcids = 'ACDEFGHIKLMNPQRSTVWY'
    label, seq = line.split()
    features = [0]*19
    for aa in seq:
        for i in range(7):
            features[i] += aa_info[aa][i]
        for i in [0, 1, 2, 3]:
            if aa_info[aa][i] > 0:
                features[7+2*i] += aa_info[aa][i]
            else: 
                features[8+2*i] += aa_info[aa][i]
        if aa_info[aa][6] > 0:
            features[15] += aa_info[aa][i]
        else: 
            features[16] += aa_info[aa][i]
        if aa in 'VILMFWC':
            features[17] += 1
        if aa in 'RK':
            features[18] += 1
    return LabeledPoint(label, features)

In [4]:
fileName = 'YeastTM20.dat'
rawData = sc.textFile(fileName, 2).map(parse)
print rawData.take(5)

[LabeledPoint(0.0, [8.9,-1.0,6.0,5.47,33.1,116.8,11.46,30.9,-22.0,2.0,-3.0,8.0,-2.0,6.88,-1.41,6.81,-1.34,5.0,2.0]), LabeledPoint(0.0, [-27.8,-6.0,9.0,2.51,54.2,106.0,8.2,11.9,-39.7,1.0,-7.0,12.0,-3.0,3.38,-0.87,3.38,-0.87,2.0,1.0]), LabeledPoint(0.0, [-46.9,-6.0,14.0,4.93,58.5,102.0,14.89,4.2,-51.1,1.0,-7.0,16.0,-2.0,5.87,-0.94,5.8,-0.87,1.0,1.0]), LabeledPoint(0.0, [-4.4,2.0,4.0,4.72,52.2,128.0,10.57,28.8,-33.2,4.0,-2.0,8.0,-4.0,7.52,-2.8,7.45,-2.73,6.0,4.0]), LabeledPoint(1.0, [33.2,0.0,4.0,-8.75,19.0,116.5,-12.04,42.1,-8.9,0.0,0.0,4.0,0.0,1.31,-10.06,1.31,-10.06,12.0,0.0])]


In [5]:
weights = [0.8, 0.1, 0.1]
seed = 1
rawTrainData, rawValidationData, rawTestData = rawData.randomSplit(weights, seed)
rawTrainData.cache()
rawValidationData.cache()
rawTestData.cache()
nTrain = rawTrainData.count()
nVal = rawValidationData.count()
nTest = rawTestData.count()
print nTrain, nVal, nTest, nTrain + nVal + nTest

26490 3326 3302 33118


Loss should calculated for a give prediction and label

In [6]:
def computeLogLoss(p, y):
    """Calculates the value of log loss for a given probabilty and label.

    Note:
        log(0) is undefined, so when p is 0 we need to add a small value (epsilon) to it
        and when p is 1 we need to subtract a small value (epsilon) from it.

    :param p (float): A probabilty between 0 and 1.
           y (int): A label.  Takes on the values 0 and 1.

    :return: float: The log loss value.
    """
    epsilon = 10e-12
    if y == 1:
        pp = p
    if y == 0:
        pp = 1-p
    if pp == 0:
        return -log(pp+epsilon)
    elif pp ==1:
        return -log(pp-epsilon)
    else:
        return -log(pp)
    
def getP(x, w, intercept):
    """ calculate the probability of a set of features to be transmembrane domain

    :param x: all the features, nparray(9)
    :param w: model weight, nparray(9)
    :param intercept: model intercept

    :return: float: the probability of a set of features to be transmembrane domain
    """
    rawPrediction = x.dot(w)+intercept

    # Bound the raw prediction value
    rawPrediction = min(rawPrediction, 20)
    rawPrediction = max(rawPrediction, -20)
    return 1/(1+exp(-rawPrediction))

In [7]:
def evaluateResults(model, data):
    """ Calculates the log loss for the data given the model.

    :param model (LogisticRegressionModel): A trained logistic regression model.
           data (RDD of LabeledPoint): Labels and features for each observation.

    :return: float: Log loss for the data.
    """
    log_loss = (data.map(lambda x: computeLogLoss(getP(x.features, model.weights, model.intercept), x.label))
                    .reduce(lambda x, y: x+y))/data.count()
    return log_loss

In [8]:
# try fixed hyperparameters
numIters = 500
stepSize = 1
regParam = 1e-6
regType = 'l2'
includeIntercept = True

model0 = LogisticRegressionWithSGD.train(rawTrainData,
                                         iterations=numIters, 
                                         step=stepSize, 
                                         miniBatchFraction=1.0, 
                                         initialWeights=None, 
                                         regParam=regParam, 
                                         regType=regType, 
                                         intercept=includeIntercept)
print model0.weights, model0.intercept

[13.196021072,0.75333899498,-0.311891614544,-5.50864829707,-1.05723381089,-1.8140446184,-8.2405744326,3.37225867775,9.82376239421,-0.89362860677,1.64696760175,-1.29629799285,0.984406378309,-2.48638062547,-3.0222676716,-2.47083324234,-3.03781505473,2.20595006412,-0.89362860677] 0.985999681109


In [9]:
classOneFracTrain = (rawTrainData.map(lambda x: x.label)
                                 .reduce(lambda x, y: x+y))/rawTrainData.count()
print classOneFracTrain

logLossTrBase = (rawTrainData.map(lambda x: x.label)
                             .map(lambda x: computeLogLoss(classOneFracTrain, x))
                             .reduce(lambda x, y: x+y))/rawTrainData.count()
print 'Baseline Train Logloss = {0:.3f}\n'.format(logLossTrBase)

0.284144960362
Baseline Train Logloss = 0.597



In [10]:
logLossTrLR0 = evaluateResults(model0, rawTrainData)
print ('Logloss:\n\tLogReg = {0:.3f}'
       .format(logLossTrLR0))

Logloss:
	LogReg = 0.806


In [11]:
logLossVa = evaluateResults(model0, rawValidationData)
print ('Logloss:\n\tLogReg = {0:.3f}'
       .format(logLossVa))

Logloss:
	LogReg = 0.788


In [12]:
numIters = 100
regType = 'l2'
includeIntercept = True

# Initialize variables using values from initial model training
bestModel = None
bestLogLoss = 1e10

stepSizes = [0.01, 0.1, 1, 10]
regParams = [1e-6, 1e-3]
for stepSize in stepSizes:
    for regParam in regParams:
        model = (LogisticRegressionWithSGD
                 .train(rawTrainData, numIters, stepSize, regParam=regParam, regType=regType,
                        intercept=includeIntercept))
        logLossVa = evaluateResults(model, rawValidationData)
        print ('\tstepSize = {0:.2f}, regParam = {1:.0e}: logloss = {2:.3f}'
               .format(stepSize, regParam, logLossVa))
        if (logLossVa < bestLogLoss):
            bestModel = model
            bestLogLoss = logLossVa

print ('Validation Logloss:\n\tBaseline = {0:.3f}\n\tLogReg = {1:.3f}'
       .format(logLossTrBase, bestLogLoss))

	stepSize = 0.01, regParam = 1e-06: logloss = 0.128
	stepSize = 0.01, regParam = 1e-03: logloss = 0.128
	stepSize = 0.10, regParam = 1e-06: logloss = 0.632
	stepSize = 0.10, regParam = 1e-03: logloss = 0.631
	stepSize = 1.00, regParam = 1e-06: logloss = 0.922
	stepSize = 1.00, regParam = 1e-03: logloss = 0.922
	stepSize = 10.00, regParam = 1e-06: logloss = 0.926
	stepSize = 10.00, regParam = 1e-03: logloss = 0.920
Validation Logloss:
	Baseline = 0.597
	LogReg = 0.128


In [13]:
logLossTe = evaluateResults(bestModel, rawTestData)
print ('Logloss:\n\tLogReg = {0:.3f}'
       .format(logLossTe))

Logloss:
	LogReg = 0.159


In [14]:
# More iteration with optimized parameters
numIters = 10000
stepSize = 0.01
regParam = 1e-6
regType = 'l2'
includeIntercept = True

model1 = LogisticRegressionWithSGD.train(rawTrainData,
                                         iterations=numIters, 
                                         step=stepSize, 
                                         miniBatchFraction=1.0, 
                                         initialWeights=None, 
                                         regParam=regParam, 
                                         regType=regType, 
                                         intercept=includeIntercept)
print model1.weights, model1.intercept

[0.127752353647,0.0248992252734,0.00743572396795,-0.103688386021,-0.00750701102293,-0.00174062161425,-0.102516810939,-0.02997609561,0.157728449257,-0.0161521904671,0.0410514157405,-0.0123707848711,0.019806508839,-0.0484790847163,-0.0552093013047,-0.0478423792262,-0.0558460067948,0.0278284300459,-0.0161521904671] 0.999916812177


In [15]:
logLossTe1 = evaluateResults(model1, rawTestData)
print ('Logloss:\n\tLogReg = {0:.3f}'
       .format(logLossTe1))

Logloss:
	LogReg = 0.104


In [16]:
for x in rawData.take(10):
    print x
    print getP(x.features, model1.weights, model1.intercept)
    print computeLogLoss(getP(x.features, model.weights, model.intercept), x.label)

(0.0,[8.9,-1.0,6.0,5.47,33.1,116.8,11.46,30.9,-22.0,2.0,-3.0,8.0,-2.0,6.88,-1.41,6.81,-1.34,5.0,2.0])
0.00589740348095
2.06115358327e-09
(0.0,[-27.8,-6.0,9.0,2.51,54.2,106.0,8.2,11.9,-39.7,1.0,-7.0,12.0,-3.0,3.38,-0.87,3.38,-0.87,2.0,1.0])
8.75642730504e-06
2.06115358327e-09
(0.0,[-46.9,-6.0,14.0,4.93,58.5,102.0,14.89,4.2,-51.1,1.0,-7.0,16.0,-2.0,5.87,-0.94,5.8,-0.87,1.0,1.0])
4.72361429188e-08
2.06115358327e-09
(0.0,[-4.4,2.0,4.0,4.72,52.2,128.0,10.57,28.8,-33.2,4.0,-2.0,8.0,-4.0,7.52,-2.8,7.45,-2.73,6.0,4.0])
0.000223313507668
2.06115358327e-09
(1.0,[33.2,0.0,4.0,-8.75,19.0,116.5,-12.04,42.1,-8.9,0.0,0.0,4.0,0.0,1.31,-10.06,1.31,-10.06,12.0,0.0])
0.996587577197
2.06115369429e-09
(1.0,[24.1,0.0,5.0,-6.57,19.0,116.7,-7.73,36.6,-12.5,0.0,0.0,5.0,0.0,2.05,-8.62,2.05,-8.62,10.0,0.0])
0.958963396286
2.06115369429e-09
(1.0,[19.4,1.0,7.0,-4.07,10.5,118.5,-5.57,30.4,-11.0,1.0,0.0,8.0,-1.0,2.22,-6.29,2.22,-6.29,10.0,1.0])
0.9028087437
2.06115369429e-09
(1.0,[19.7,0.0,10.0,-5.28,10.7,113.5,-8.4

In [29]:
print model0.weights

[0.197843553366,0.0145948882501,-0.0100536691748,-0.0821223158907,-0.0167367335816,-0.0405236595803,-0.118249079876,0.0304857073854,-0.0119070274029]
